# WINDOW FUNCTION GSL PERIODOGRAMS - RV BATCH FILE PROCESSING

In [1]:
# NOTA IMPORTANTE - CÓMO IMPLEMENTAR LO DEL "TIMEOUT".
# https://stackoverflow.com/questions/25027122/break-the-function-after-certain-time
# NO SIRVE - SOLO SIRVE PARA ENTORNOS UNIX

This notebook calculates the periodograms for the Window Functions of the RV curves. It saves a simple text file with the periodogram result and the figure of the `gls` object. Notice that, in this case, the error for the window function "radial velocities" values is set to a constant value of 0.001 (so as the weighting of the periodogram has no impact).

Error control is implemented to prevent exceptions and timeouts to interrupt the processing. In case any of these errors happen, it is so recorded in the results table. Timeout value is dinamically updated, as the calculation for each record is completed.

**NOTE:** there seems to be some kind of memory leak or something similar, because the browser crashes if you try to execute all loop iterations at once. Hence, it is necessary to execute the loop in batches of 100 elements, for example.

**Calculation conditions:**

The stars currently under analysis are in the $He_{3}$ instability band, so they are expected to have effective temperatures in the range $T_{eff}\in[3300, 4300] K$, $\log g\in[4.5, 5.1]$, and masses in the range $M_{star}\in[0.20, 0.60] M_{Sun}$.

According to _Table 2_ in _The theoretical instability strip of M dwarf stars (Rodríguez-López, C., et al. 2014, MNRAS, 438, 2371)_ these stars have typical periods of $20 min$ to $3 h$, so we will set the limits of the periodogram frequencies for periods between $5 min$ and $10 h$. In days, this corresponds to a range of $P\in[0.003472, 0.416667] d$ or, equivalently, frequencies in the range $f\in[288.0, 2.4] d^{-1}$.

We will use the _Generalized Lomb Scargle Periodogram_ method, as described in [Zechmeister and Kürster, 2009](https://www.aanda.org/articles/aa/full_html/2009/11/aa11296-08/aa11296-08.html) and implemented by GitHub repository [mzechmeister/GLS](https://github.com/mzechmeister/GLS), using the default _ZK_ normalization.

## Modules and configuration

### Modules

In [2]:
# Modules import:
#from collections import OrderedDict
import pandas as pd
import numpy as np
import time

import warnings

from scipy import stats

# https://github.com/mzechmeister/GLS
from gls import Gls

from astropy.table import Table, QTable
#from astropy.timeseries import TimeSeries
from astropy import units as u
from astropy.io import fits
#from astropy.time import Time

import lightkurve as lk

#%matplotlib inline
import matplotlib.pyplot as plt

from pylab import rcParams
rcParams['figure.figsize'] = 11, 11

#import seaborn as sns
#sns.set_style("white", {'figure.figsize':(15,10)})
#sns.set_style("whitegrid")
#sns.set(rc={'figure.figsize':(15,8)})

### Configuration

In [3]:
# Configuration:
# Files and folders (WARNING: THIS FOLDER STRUCTURE MUST EXIST PREVIOUSLY):
GTO_FILE = "../data/CARM_VIS_objects_with_PG_and_WF.csv" # NOTE: initially this should be a copy of the previous file.
IN_RV_FOLDER = "../data/CARM_VIS_RVs/"
OUT_IMG_FOLDER = "../data/CARM_VIS_RVs_WinFunc_PGs/figures/"
OUT_PROCESSED_FOLDER = "../data/CARM_VIS_RVs_WinFunc_PGs/"
#OUT_GTO_FILE = "../data/GTO_objects_withRVPG.csv"
#OUT_PG_FILE = "../data/GTO_PGs.csv"
#OUT_ERROR_FILE = "../data/GTO_PG_ERRORs.csv"

# Periodogram constants:
FBEG = 2.4 # d^{-1}, corresponds to a period P=10 hours
FEND = 288 # d^{-1}, corresponds to a period P=5 min
NUM_POINTS = 2856001 # 2856000 for RV time-series
PBEG = None # Default
PEND = None # Default
OFAC = 10 # Default
HIFAC = 1 # Default
#FREQ = np.linspace(start=FBEG, stop=FEND, num=1000001) # 1,000,001 points between 4.8 and 144 d^(-1)
FREQ = np.linspace(start=FBEG, stop=FEND, num=NUM_POINTS)
# Must be compatible with FBEG, FEND values
NORM = "ZK" # Default
LS = False # Default
FAST = False # Default
VERBOSE = False # Default
FAP_LEVELS_PLOT = [0.01, 0.05, 0.10] # FAP reference levels to plot

# Initial timeout value (to prevent a stuck file to interrupt the whole process)
#INITIAL_TIMEOUT = 600 # Seconds


### Functions

In [4]:
def rv_load(filename: str):
    '''Load the RV file and returns a lightcurve object'''
    rv_lk = Table.read(filename, format='ascii',
                    names = ['time', 'RV', 'eRV'], units=[u.d, u.meter / u.second, u.meter / u.second])
    rv_lk = lk.LightCurve(time=rv_lk['time'], flux=rv_lk['RV'], flux_err=rv_lk['eRV'])
    return rv_lk

## Data processing

### GTO data loading

In [6]:
# Load GTO data table:
gto = pd.read_csv(GTO_FILE, sep=',', decimal='.')
gto.head(5)

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_offset_PG_TESS,WF_e_offset_PG_TESS,WF_FAP_PG_TESS,WF_valid_PG_TESS,WF_error_PG_TESS,WF_elapsed_time_PG_TESS,WF_plain_file_TESS,WF_fig_file_TESS,PG_file_RV,PG_file_TESS
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000997,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23585+076_RV_PG.dat,NaN
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.000998,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23556-061_RV_PG.dat,NaN
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,999.999756,2.151008e-06,1.0,1.0,NaN,94.758838,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23548+38...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23548+385_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23548+385_RV_PG.dat
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,1000.000122,9.022946e-07,1.0,1.0,NaN,132.607176,../data/CARM_VIS_TESS_WinFunc_PGs/WF_J23505-09...,../data/CARM_VIS_TESS_WinFunc_PGs/figures/WF_J...,../data/CARM_VIS_RVs_PGs/J23505-095_RV_PG.dat,../data/CARM_VIS_TESS_PGs/J23505-095_RV_PG.dat
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,NaN,NaN,NaN,0.0,Not recognized as a supported data product:\nn...,0.001995,NaN,NaN,../data/CARM_VIS_RVs_PGs/J23492+024_RV_PG.dat,NaN


In [7]:
gto.shape

(390, 300)

In [8]:
print(list(gto.columns))

['Karmn', 'Name', 'Comp', 'GJ', 'RA_J2016_deg', 'DE_J2016_deg', 'RA_J2000', 'DE_J2000', 'l_J2016_deg', 'b_J2016_deg', 'Ref01', 'SpT', 'SpTnum', 'Ref02', 'Teff_K', 'eTeff_K', 'logg', 'elogg', '[Fe/H]', 'e[Fe/H]', 'Ref03', 'L_Lsol', 'eL_Lsol', 'Ref04', 'R_Rsol', 'eR_Rsol', 'Ref05', 'M_Msol', 'eM_Msol', 'Ref06', 'muRA_masa-1', 'emuRA_masa-1', 'muDE_masa-1', 'emuDE_masa-1', 'Ref07', 'pi_mas', 'epi_mas', 'Ref08', 'd_pc', 'ed_pc', 'Ref09', 'Vr_kms-1', 'eVr_kms-1', 'Ref10', 'ruwe', 'Ref11', 'U_kms-1', 'eU_kms-1', 'V_kms-1', 'eV_kms-1', 'W_kms-1', 'eW_kms-1', 'Ref12', 'sa_m/s/a', 'esa_m/s/a', 'Ref13', 'SKG', 'Ref14', 'SKG_lit', 'Ref14_lit', 'Pop', 'Ref15', 'vsini_flag', 'vsini_kms-1', 'evsini_kms-1', 'Ref16', 'P_d', 'eP_d', 'Ref17', 'pEWHalpha_A', 'epEWHalpha_A', 'Ref18', 'log(LHalpha/Lbol)', 'elog(LHalpha/Lbol)', 'Ref19', '1RXS', 'CRT_s-1', 'eCRT_s-1', 'HR1', 'eHR1', 'HR2', 'eHR2', 'Flux_X_E-13_ergcm-2s-1', 'eFlux_X_E-13_ergcm-2s-1', 'LX/LJ', 'eLX/LJ', 'Ref20', 'Activity', 'Ref21', 'FUV_mag',

### Add columns for new calculation data

Generate the proper auxiliary columns (with the basic periodograms results for window function).

In [9]:
# Additional columns:
if 'WF_maxP_PG_RV' in gto.columns:
    pass
else:
    gto['WF_maxP_PG_RV'] = None # Max power value in the PG
    gto['WF_maxSNR_PG_RV'] = None # Max power value in the PG
    gto['WF_rms_PG_RV'] = None # RMS value in the PG residuals
    gto['WF_f_PG_RV'] = None # Best frequency in the PG (d^(-1))
    gto['WF_e_f_PG_RV'] = None # Error of the best frequency in the PG (d^(-1))
    gto['WF_Pd_PG_RV'] = None # Best period in the PG (d)
    gto['WF_e_Pd_PG_RV'] = None # Error of the best period in the PG (d)
    gto['WF_Ph_PG_RV'] = None # Best period in the PG (hours)
    gto['WF_e_Ph_PG_RV'] = None # Error of the best period in the PG (hours)
    gto['WF_Pm_PG_RV'] = None # Best period in the PG (minutes)
    gto['WF_e_Pm_PG_RV'] = None # Error of the best period in the PG (minutes)
    gto['WF_A_PG_RV'] = None # Amplitude of the best frequency
    gto['WF_e_A_PG_RV'] = None # Error of the amplitude of the best frequency
    gto['WF_ph_PG_RV'] = None # Amplitude of the best frequency
    gto['WF_e_ph_PG_RV'] = None # Error of the amplitude of the best frequency
    gto['WF_T0_PG_RV'] = None # Reference epoch of the best frequency
    gto['WF_e_T0_PG_RV'] = None # Error of the epoch of the best frequency
    gto['WF_offset_PG_RV'] = None # Offset of the best frequency
    gto['WF_e_offset_PG_RV'] = None # Error of the offset of the best frequency
    gto['WF_FAP_PG_RV'] = None # False alarm probability
    gto['WF_valid_PG_RV'] = None # Flag to indicate if the periodogram calculation succeeded (1) or not (0).
    gto['WF_error_PG_RV'] = None # The error raised during processing. Empty if processing was successful.
    gto['WF_elapsed_time_PG_RV'] = None # The time elapsed in calculation
    gto['WF_plain_file_RV'] = None # The name of the processed file.
    gto['WF_fig_file_RV'] = None # The name of the figure file.


### Batch processing of all RV files

In [10]:
n = len(gto)
n

390

#### Calculate the observation time span

In [24]:
obs_start = None
obs_end = None
for i in range(0, len(gto)):
    try:
        karmn = gto.loc[i, 'Karmn'] # CARMENES name
        # load start time series:
        rv_file = gto.loc[i, 'rv_file']
        rv_lk = rv_load(rv_file)
        if obs_start is None:
            obs_start = rv_lk['time'].min()
        else:
            obs_start = min(obs_start, rv_lk['time'].min())
        if obs_end is None:
            obs_end = rv_lk['time'].max()
        else:
            obs_end = max(obs_end, rv_lk['time'].max())
    except:
        print("WARNING - Some problem happened with star %s, skipping..." %karmn)

print("RESULTS:")
print("Start observation time: %s" %obs_start)
print("End observation time: %s" %obs_end)
print("survey time span: %s" %(obs_end - obs_start))

WARNING - Some problem happened with star J17572+707, skipping...
WARNING - Some problem happened with star J16102-193, skipping...
WARNING - Some problem happened with star J04173+088, skipping...
RESULTS:
Start observation time: 2457392.299527406
End observation time: 2459687.681815571
survey time span: 2295.382288164925


In [31]:
(rv_lk['time'][1:] - rv_lk['time'][:-1]).min()

<TimeDelta object: scale='tdb' format='jd' value=0.9477409883402288>

In [25]:
obs_start

<Time object: scale='tdb' format='jd' value=2457392.299527406>

In [26]:
obs_start + 1.5

<Time object: scale='tdb' format='jd' value=2457393.799527406>

In [27]:
obs_end - obs_start

<TimeDelta object: scale='tdb' format='jd' value=2295.382288164925>

#### Batch processing

**Note:** previous data (not NZP-corrected) failed on indexes: 99, 273 and 344.

In [10]:
warnings.filterwarnings('ignore')# Batch processing:
lapse_list = []
median_lapse = None
# NOTE THE FOR LOOP IS SEPARATED INTO SEVERAL "BATCHES", SO AS TO PREVENT POTENTIAL MEMORY PROBLEMS
# OR TOO MUCH TIME ELAPSED
#for i in range(0, 3): # TEST
#for i in [154, 251, 378]: # To repeat the calculation for specific objects.
#for i in range(0, len(gto)): # ALL RECORDS
#for i in range(0, 96):
#for i in range(96, 197):
#for i in range(197, 293):
for i in range(293, len(gto)):
    start_time = time.time()
    # Names:
    karmn = gto.loc[i, 'Karmn'] # CARMENES name
    commn = gto.loc[i, 'Name'] # Common name
    tic_id = str(gto.loc[i, 'TIC_id']) # TESS TIC identifier
    print("Record: %d, window function PG calculation started at %s"
          %(i, time.strftime('%d/%m/%Y, %H:%M:%S', time.localtime(start_time))))
    if median_lapse is None:
        print("Previous median lapse time: %s" %median_lapse)
    else:
        print("Previous median lapse time: %.2f seconds" %median_lapse)
    print("Processing %s star..." %karmn)
    #if True:
    try:
        # LOAD RV FILE:
        rv_file = gto.loc[i, 'rv_file']
        print("filename: %s" %rv_file)
        rv_lk = rv_load(rv_file)
        
        # GENERATE THE WINDOW FUNCTION
        # All values to 1.0 and errors to 0.0,time points are the same.
        #rv_lk['flux'].fill(1.0 * u.m / u.s)
        #rv_lk['flux_err'].fill(0.001 * u.m / u.s)
        
        
        # GENERATE PERIODOGRAM:
        gls = Gls((rv_lk['time'].value, rv_lk['flux'].value, rv_lk['flux_err'].value),
              fbeg=FBEG, fend=FEND, Pbeg=PBEG, Pend=PEND, ofac=OFAC, hifac=HIFAC, freq=FREQ,
              norm=NORM, ls=LS, fast=FAST, verbose=VERBOSE)

        # FILL IN THE DATA IN THE GTO TABLE:
        gto.loc[i, 'WF_maxP_PG_RV'] = gls.power.max()
        gto.loc[i, 'WF_maxSNR_PG_RV'] = gls.best['amp'] / gls.rms
        gto.loc[i, 'WF_rms_PG_RV'] = gls.rms
        gto.loc[i, 'WF_f_PG_RV'] = gls.best['f']
        gto.loc[i, 'WF_e_f_PG_RV'] = gls.best['e_f']
        gto.loc[i, 'WF_Pd_PG_RV'] = gls.best['P']
        gto.loc[i, 'WF_e_Pd_PG_RV'] = gls.best['e_P']
        gto.loc[i, 'WF_Ph_PG_RV'] = 24.0 * gls.best['P']
        gto.loc[i, 'WF_e_Ph_PG_RV'] = 24.0 * gls.best['e_P']
        gto.loc[i, 'WF_Pm_PG_RV'] = 24.0 * 60.0 * gls.best['P']
        gto.loc[i, 'WF_e_Pm_PG_RV'] = 24.0 * 60.0 * gls.best['e_P']
        gto.loc[i, 'WF_A_PG_RV'] = gls.best['amp']
        gto.loc[i, 'WF_e_A_PG_RV'] = gls.best['e_amp']
        gto.loc[i, 'WF_ph_PG_RV'] = gls.best['ph']
        gto.loc[i, 'WF_e_ph_PG_RV'] = gls.best['e_ph']
        gto.loc[i, 'WF_T0_PG_RV'] = gls.best['T0']
        gto.loc[i, 'WF_e_T0_PG_RV'] = gls.best['e_T0']
        gto.loc[i, 'WF_offset_PG_RV'] = gls.best['offset']
        gto.loc[i, 'WF_e_offset_PG_RV'] = gls.best['e_offset']
        gto.loc[i, 'WF_FAP_PG_RV'] = gls.FAP(Pn=None)
        
        # SAVE THE DATA:
        # (No'fits' file here, we just use the 'toFile' methoc of 'gls' instance)
        plain_filename = OUT_PROCESSED_FOLDER + "WF_" + karmn + "_RV_PG.dat"
        gls.toFile(plain_filename, header=False)
        
        # GENERATE THE FIGURE:
        fig = gls.plot(block=False, period=False,
                       fap=FAP_LEVELS_PLOT, gls=True, data=True, residuals=True)
        figure_title = "Window Function PG for %s, P=%.4f [min], FAP=%.4f%%" \
            %(karmn, gto.loc[i, 'WF_Pm_PG_RV'], 100.0 * gto.loc[i, 'WF_FAP_PG_RV'])
        fig.suptitle(figure_title, fontdict = {'fontsize' : 36})
        fig.tight_layout()
        # Save the figure to disk:
        fig_file = OUT_IMG_FOLDER + "WF_" + karmn + "_RV_PG.jpg"
        fig.savefig(fig_file)
        plt.close() # Prevent the figure from showing.

        # SET THE RECORD CALCULATION AS VALID AND STORE THE REULTING FILENAMES
        gto.loc[i, 'WF_valid_PG_RV'] = 1
        gto.loc[i, 'WF_error_PG_RV'] = ""
        gto.loc[i, 'WF_plain_file_RV'] = plain_filename
        gto.loc[i, 'WF_fig_file_RV'] = fig_file

        # UPDATE THE AVERAGE RECORD PROCESSING TIME:
        lapse = time.time() - start_time
        lapse_list.append(lapse)
        median_lapse = np.nanmedian(lapse_list)
        gto.loc[i, 'WF_elapsed_time_PG_RV'] = lapse
        
        # SAVE THE UPDATED GTO TABLE TO DISK:
        gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        
        # Report successful execution:
        print("Elapsed time: %.2f seconds" %lapse)
        print("... SUCCESS.")
        
    #else:
    except Exception as e:
        # Some error happened, establish the record as not valid and record the error:
        error = "*** Some ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, e)
        print(error)
        gto.loc[i, 'WF_valid_PG_RV'] = 0
        gto.loc[i, 'WF_error_PG_RV'] = e
        
        # Try to update the record, and save the file:
        try:
            # UPDATE THE AVERAGE RECORD PROCESSING TIME:
            lapse = time.time() - start_time
            lapse_list.append(lapse)
            median_lapse = np.nanmedian(lapse_list)
            gto.loc[i, 'WF_elapsed_time_PG_RV'] = lapse
            print("Elapsed time: %.2f seconds" %lapse)

            # SAVE THE UPDATED GTO TABLE TO DISK:
            gto.to_csv(GTO_FILE, sep=',', decimal='.', index=False)
        except Exception as e2:
            error_2 = "*** Additional ERROR happened with record #%d, %s star. Error=%s" %(i, karmn, str(e2))
            print(error)
            gto.loc[i, 'WF_error_PG_RV'] = gto.loc[i, 'WF_error_PG_RV'] + "/" + str(e2)



Record: 293, window function PG calculation started at 07/08/2022, 15:15:21
Previous median lapse time: None
Processing J06246+234 star...
filename: ../data/CARM_VIS_RVs/J06246+234.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J06246+234_RV_PG.dat
Elapsed time: 29.41 seconds
... SUCCESS.
Record: 294, window function PG calculation started at 07/08/2022, 15:15:51
Previous median lapse time: 29.41 seconds
Processing J06105-218 star...
filename: ../data/CARM_VIS_RVs/J06105-218.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J06105-218_RV_PG.dat
Elapsed time: 32.09 seconds
... SUCCESS.
Record: 295, window function PG calculation started at 07/08/2022, 15:16:23
Previous median lapse time: 30.75 seconds
Processing J06103+821 star...
filename: ../data/CARM_VIS_RVs/

Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J05084-210_RV_PG.dat
Elapsed time: 30.28 seconds
... SUCCESS.
Record: 312, window function PG calculation started at 07/08/2022, 15:25:25
Previous median lapse time: 31.08 seconds
Processing J05062+046 star...
filename: ../data/CARM_VIS_RVs/J05062+046.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J05062+046_RV_PG.dat
Elapsed time: 29.99 seconds
... SUCCESS.
Record: 313, window function PG calculation started at 07/08/2022, 15:25:55
Previous median lapse time: 30.86 seconds
Processing J05033-173 star...
filename: ../data/CARM_VIS_RVs/J05033-173.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J05033-173_RV_PG.dat
Elapsed time: 31.23 seconds
... SUCCESS.
Record: 314, window function PG calc

No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J04219+213_RV_PG.dat
Elapsed time: 28.81 seconds
... SUCCESS.
Record: 330, window function PG calculation started at 07/08/2022, 15:34:44
Previous median lapse time: 30.28 seconds
Processing J04198+425 star...
filename: ../data/CARM_VIS_RVs/J04198+425.avc.dat
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J04198+425_RV_PG.dat
Elapsed time: 31.44 seconds
... SUCCESS.
Record: 331, window function PG calculation started at 07/08/2022, 15:35:16
Previous median lapse time: 30.35 seconds
Processing J04173+088 star...
filename: ../data/CARM_VIS_RVs/J04173+088.avc.dat
*** Some ERROR happened with record #331, J04173+088 star. Error=zero-size array to reduction operation minimum which has no identity
Elapsed time: 0.00 seconds
Record: 332, window function PG calculation started at 07/08/2022, 15:35:16
Previous median 

Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J02530+168_RV_PG.dat
Elapsed time: 46.32 seconds
... SUCCESS.
Record: 349, window function PG calculation started at 07/08/2022, 15:44:17
Previous median lapse time: 30.35 seconds
Processing J02519+224 star...
filename: ../data/CARM_VIS_RVs/J02519+224.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J02519+224_RV_PG.dat
Elapsed time: 29.89 seconds
... SUCCESS.
Record: 350, window function PG calculation started at 07/08/2022, 15:44:46
Previous median lapse time: 30.28 seconds
Processing J02489-145W star...
filename: ../data/CARM_VIS_RVs/J02489-145W.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J02489-145W_RV_PG.dat
Elapsed time: 30.00 seconds
... SUCCESS.
Record: 351, window function PG c

Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J01550+379_RV_PG.dat
Elapsed time: 28.60 seconds
... SUCCESS.
Record: 367, window function PG calculation started at 07/08/2022, 15:53:25
Previous median lapse time: 30.23 seconds
Processing J01518+644 star...
filename: ../data/CARM_VIS_RVs/J01518+644.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J01518+644_RV_PG.dat
Elapsed time: 29.57 seconds
... SUCCESS.
Record: 368, window function PG calculation started at 07/08/2022, 15:53:55
Previous median lapse time: 30.17 seconds
Processing J01433+043 star...
filename: ../data/CARM_VIS_RVs/J01433+043.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J01433+043_RV_PG.dat
Elapsed time: 30.54 seconds
... SUCCESS.
Record: 369, window function PG calc

Elapsed time: 41.46 seconds
... SUCCESS.
Record: 385, window function PG calculation started at 07/08/2022, 16:03:02
Previous median lapse time: 30.35 seconds
Processing J00183+440 star...
filename: ../data/CARM_VIS_RVs/J00183+440.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J00183+440_RV_PG.dat
Elapsed time: 42.96 seconds
... SUCCESS.
Record: 386, window function PG calculation started at 07/08/2022, 16:03:45
Previous median lapse time: 30.41 seconds
Processing J00162+198W star...
filename: ../data/CARM_VIS_RVs/J00162+198W.avc.dat
No output of frequency error.
Increase frequency range to sample the peak maximum.
Results have been written to file:  ../data/CARM_VIS_RVs_WinFunc_PGs/WF_J00162+198W_RV_PG.dat
Elapsed time: 28.73 seconds
... SUCCESS.
Record: 387, window function PG calculation started at 07/08/2022, 16:04:14
Previous median lapse time: 30.35 seconds
Processi

**Important note:** a lot of stars ($\approx4/5$) have given an error of the type "No output of frequency error". We should look more carefully into this issue, taking one of those stars, creating the `gls` instance for its window function and tracing back the values of its attributes as per the `gls` code.

## Check the calculations that took longer to complete

We now check the calcultions that took longer, so as to try to repeat them (as it could be due to a problem with the computer itself - for example, the program stopped when the computer went into "sleep" mode).

In [12]:
gto[['Karmn', 'WF_elapsed_time_PG_RV']].sort_values(by='WF_elapsed_time_PG_RV', ascending=False) \
    .head(10)

,Karmn,WF_elapsed_time_PG_RV
281,J07274+052,75.537251
99,J17578+046,69.648498
14,J22565+165,65.651121
207,J11033+359,60.470278
288,J06548+332,54.218976
163,J13299+102,52.280427
4,J23492+024,52.214123
348,J02530+168,46.318784
53,J20260+585,46.117714
160,J13457+148,44.760722


## Summary of calculated periodograms

Number of objects with RV PG properly calculated:

In [13]:
gto[gto['WF_valid_PG_RV'] == 1.0]

,Karmn,Name,Comp,GJ,RA_J2016_deg,DE_J2016_deg,RA_J2000,DE_J2000,l_J2016_deg,b_J2016_deg,...,WF_T0_PG_RV,WF_e_T0_PG_RV,WF_offset_PG_RV,WF_e_offset_PG_RV,WF_FAP_PG_RV,WF_valid_PG_RV,WF_error_PG_RV,WF_elapsed_time_PG_RV,WF_plain_file_RV,WF_fig_file_RV
0,J23585+076,Wolf 1051,AB,4383,359.636420,7.656947,23:58:32.65,+07:39:30.1,100.839686,-52.931049,...,2.457588e+06,NaN,1.0,NaN,NaN,1.0,NaN,33.078205,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J23585+076...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J2...
1,J23556-061,GJ 912,AB,912,358.913617,-6.144283,23:55:39.78,-06:08:33.4,88.129933,-65.175491,...,2.457594e+06,NaN,1.0,NaN,NaN,1.0,NaN,29.850991,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J23556-061...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J2...
2,J23548+385,RX J2354.8+3831,-,NaN,358.713658,38.526340,23:54:51.46,+38:31:36.2,110.941908,-23.024449,...,2.457593e+06,NaN,1.0,NaN,NaN,1.0,NaN,28.539767,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J23548+385...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J2...
3,J23505-095,LP 763-012,-,4367,357.634705,-9.560964,23:50:31.64,-09:33:32.7,80.777067,-67.303426,...,2.457595e+06,NaN,1.0,NaN,NaN,1.0,NaN,32.203607,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J23505-095...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J2...
4,J23492+024,BR Psc,-,908,357.306604,2.396918,23:49:12.53,+02:24:04.4,93.567467,-56.885396,...,2.457395e+06,NaN,1.0,NaN,-5.559949e-114,1.0,NaN,52.214123,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J23492+024...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,J00183+440,GX And,A,15A,4.613226,44.024787,00:18:22.88,+44:01:22.6,116.690592,-18.446865,...,2.457395e+06,NaN,1.0,NaN,NaN,1.0,,42.959178,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J00183+440...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J0...
386,J00162+198W,EZ Psc,Aab,1006A,4.064321,19.857041,00:16:14.63,+19:51:37.5,111.729451,-42.248173,...,2.457592e+06,NaN,1.0,NaN,NaN,1.0,,28.726722,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J00162+198...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J0...
387,J00162+198E,LP 404-062,B,1006B,4.070651,19.860692,00:16:16.15,+19:51:50.5,111.738263,-42.245679,...,2.457608e+06,NaN,1.0,NaN,NaN,1.0,,29.341945,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J00162+198...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J0...
388,J00067-075,GJ 1002,-,1002,1.676350,-7.546475,00:06:43.20,-07:32:17.0,92.444693,-67.730511,...,2.457596e+06,NaN,1.0,NaN,-3.648325e+31,1.0,,33.967957,../data/CARM_VIS_RVs_WinFunc_PGs/WF_J00067-075...,../data/CARM_VIS_RVs_WinFunc_PGs/figures/WF_J0...


There are three stars for which periodogram calculation failed. It was verified that in all these cases, the RV file is empty, with no data.

In [14]:
gto[gto['WF_valid_PG_RV'] == 0.0][['Karmn', 'rv_file', 'WF_valid_PG_RV', 'WF_error_PG_RV']]

,Karmn,rv_file,WF_valid_PG_RV,WF_error_PG_RV
100,J17572+707,../data/CARM_VIS_RVs/J17572+707.avc.dat,0.0,zero-size array to reduction operation minimum...
126,J16102-193,../data/CARM_VIS_RVs/J16102-193.avc.dat,0.0,zero-size array to reduction operation minimum...
331,J04173+088,../data/CARM_VIS_RVs/J04173+088.avc.dat,0.0,zero-size array to reduction operation minimum...


# Summary

**OBSERVATIONS AND CONCLUSIONS:**
- We have completed the basic GLS periodogram calculation for the window functions of radial velocity (RV) curves of CARM_VIS GTO objetcs (NZP-corrected), and stored the results.
- Out of the 390 initial objects:
  - 387 stars had their RV periodogram calculated correctly.
  - 3 of the objects yielded errors. In all three cases, the reason was that the RV file was empty, no RV points were present:
    - J17572+707
    - J16102-193
    - J04173+088
  - However, most of the objects ($\approx 4/5$) yielded empty periodograms and the warning or error: _"WARNING: Highest peak is at the edge of the frequency range. No output of frequency error. Increase frequency range to sample the peak maximum"_. We should try to know the reason for this.